# resultsVol_1: Getting data

Anish ran synapse detection on the chessboard dataset and has provided locations of potential synapses.
Here, we pull data from those locations and export it into a nice dataset for further analysis.

In [34]:
import csv
import ndio
import numpy as np
import h5py
import scipy.linalg
from numpy import genfromtxt
import ndio.remote.neurodata as neurodata
import os
import sys

nd = neurodata(hostname = "synaptomes.neurodata.io")
local = False

TOKEN1 = "kristina15"##(sys.argv[1])##"Ex10R55"
#INPUT = "./LocationData/Ex12R75/resultVol_1.csv"##(sys.argv[2])
INPUT = "K15F0_rscaled317samp5e3_hmc5NN.csv" # Locations from meda hmc
NAME = "synaptograms"##(sys.argv[3])

bf = 5

output = TOKEN1 + "rscaled317samp5000_" + NAME + "11cubes5NN_test7"+ ".h5"
output

'kristina15rscaled317samp5000_synaptograms11cubes5NN_test7.h5'

### User defined functions

In [14]:
execfile("/Users/JLP/neurodata/synaptome-stats/Code/Python/functions.py")

In [15]:
public_tokens = nd.get_public_tokens()

In [16]:
TOKEN1 in public_tokens # Should *definitely* be true

False

In [17]:
size = nd.get_image_size(TOKEN1)
channels = nd.get_channels(TOKEN1)

xGlobal = [0,size[0]]
yGlobal = [0,size[1]]
zGlobal = [0,size[2]]

globalMaxs = [size[0], size[1], size[2]]
globalMaxs

[1826, 12986, 41]

In [28]:
chan = []
for key in channels:
    chan.append(key)
del chan[3]
del chan[-5]

## Import resultsVol_1
Import the location data from "resultVol_1.csv" making sure that each location is not too 
close to the boundary of the data and round to nearest integer.
And that the points are far enough away from the boundary.

In [29]:
loc = genfromtxt(INPUT, delimiter=',', skip_header = 1).tolist()

L1 = []
for i in range(len(loc)):
    if inRange(loc[i], globalMaxs, bf):
        L1.append(loc[i])
        
np.random.seed(317)
## Subtract one to account for python indexing from 0
LL = np.around(np.asarray(L1) - 1, decimals = 0).astype(int)
L = LL
L

array([[  400,  5723,     5],
       [  776,  5935,    15],
       [ 1152, 12900,     9],
       [  907,  4126,    15],
       [  562,  5968,    27],
       [  739,  5644,    24],
       [  260, 10960,    23],
       [  246,  9453,    31],
       [ 1011,  1946,    32],
       [  241,  9051,    32],
       [  645,  6565,    11],
       [  265,  6733,    34],
       [  910,  1011,    29],
       [  961,  4994,    19],
       [  348,  7560,    21],
       [  155,  8274,     9],
       [   84,  1749,    32],
       [  768,  7957,    25],
       [ 1337,  5763,    19],
       [   48,  5346,    26],
       [  291,  1776,    11],
       [  920,  5499,    18],
       [  363, 12876,    18],
       [  150,  9935,    23],
       [  393,  3639,    34],
       [ 1197,  5988,    28],
       [  707,  2376,    10],
       [ 1232,  4329,    16],
       [ 1251,  5659,    12],
       [  510,  2590,    19],
       [  616,  7436,     6],
       [  937, 12562,     8],
       [  203, 12174,    27],
       [  

In [30]:
xLocal = [0, max(L[:,0])]
yLocal = [0, max(L[:,1])] 
zLocal = [0, max(L[:,2])] 
len(L)

40

## Extract 11x11x11 cube around each location 
### 11x11x11 cube is used for features F0-F3, with the buffer needed for features F4-F5
then cast to vector of length 11^3 and output as an array dim = (n,1331)

#### N.B. This does take a while

In [31]:
synCube = []
for i in range(len(L)):
    print i, "out of ", len(L)-1
    cubeCh = []
    for ch in chan:
        xLocal = L[i][0]
        yLocal = L[i][1]
        zLocal = L[i][2]
        queryGlobal = { 
            'token': TOKEN1,
            'channel': ch,
            'x_start': xLocal - bf,
            'x_stop': xLocal + bf +1,
            'y_start': yLocal - bf,
            'y_stop': yLocal + bf +1,
            'z_start': zLocal - bf,
            'z_stop': zLocal + bf + 1,
            'resolution': 0   
            }
        
        gg = nd.get_cutout(**queryGlobal)
        cubeCh.append(gg)
        out = np.asarray(cubeCh)
    synCube.append(out)

0 out of  39
1 out of  39
2 out of  39
3 out of  39
4 out of  39
5 out of  39
6 out of  39
7 out of  39
8 out of  39
9 out of  39
10 out of  39
11 out of  39
12 out of  39
13 out of  39
14 out of  39
15 out of  39
16 out of  39
17 out of  39
18 out of  39
19 out of  39
20 out of  39
21 out of  39
22 out of  39
23 out of  39
24 out of  39
25 out of  39
26 out of  39
27 out of  39
28 out of  39
29 out of  39
30 out of  39
31 out of  39
32 out of  39
33 out of  39
34 out of  39
35 out of  39
36 out of  39
37 out of  39
38 out of  39
39 out of  39


In [25]:
len(chan)

4

In [32]:
#columnDat = np.transpose(np.asarray(cols))
columnNames = [str(key) for key in chan]
columnNames

['CR1_2ndA',
 'GluR2_2ndA',
 'PV_1stA',
 'VAChT_4thA',
 'GABARa1_4thA',
 'VGluT2_2ndA',
 'tubulin_8thA',
 'VGluT3_1stA',
 'TH_5thA',
 'VGluT1_3rdA',
 'NR2B_9thA',
 'VGluT1_8thA',
 'synapsinR_7thA',
 'GAD_6thA',
 'VGAT_5thA',
 'Synpod_3rdA',
 'synapsinGP_5thA',
 'psd_8thA',
 'dapi_1stA',
 'GABABR1_3rdA',
 'NMDAR1_6thA',
 'NOS_9thA',
 'gephyrin_1stA']

## Save as HDF5

In [35]:
h5fOUT = h5py.File(output, 'a')
#h5fOUT.create_dataset(TOKEN1, data=cols)
h5fOUT.create_dataset(TOKEN1, data=np.transpose(synCube))
h5fOUT.create_dataset('Locations', data=np.transpose(L))
h5fOUT.create_dataset('Channels', data=columnNames)
h5fOUT.close()